In [ ]:
import os
import sys
import timeit
import numpy
from keras.models import *
from keras.layers.core import *
from keras.layers.embeddings import *
from keras.regularizers import l2
from keras import backend as K
from scipy.io import loadmat
from scipy.io import savemat
from keras.models import model_from_json
from IPython.display import SVG
from keras.utils.visualize_util import to_graph
from keras.callbacks import ModelCheckpoint
import theano
import theano.tensor as T
from theano.tensor.shared_randomstreams import RandomStreams
to_path = "./"

In [ ]:
# Matrix of word2vec activations for each term in our dictionary, (n_terms, 300)
term_matrix = loadmat(to_path + "t1_termatrix.mat", variable_names = "target").get("target").astype("float32")
term_matrix.shape

In [ ]:
# Pretrain W2
# W2 is pretrained by autoencoding with the formula le' = sigmoid(le dot W2) dot t(W2)
from dA import dA
# The dA class from deeplearning.net was modified for this purpose.  
# In particular, the changes are:
#     1. Biases are taken out. 
#     2. The visible layer activation function is changed from sigmoid(sigmoid(le dot W2) dot t(W2))
#     3. mse is used as the cost function

index = T.lscalar()    
x = T.matrix('x')
rng = numpy.random.RandomState(123)
theano_rng = RandomStreams(rng.randint(2 ** 30))
pretrainer = dA(input = x, numpy_rng = rng, 
                theano_rng = theano_rng, 
                n_visible = 300, n_hidden = 128)
cost, updates = pretrainer.get_cost_updates(
        corruption_level=0,
        learning_rate=0.01
    )
train_data = theano.shared(name = "trainer", 
                           value = term_matrix, 
                           borrow = True)
batch_size = 10
train_da = theano.function(
        [index],
        cost,
        updates=updates,
        givens={
            x: train_data[index * batch_size: (index + 1) * batch_size]
        }
    )

In [ ]:
# Ultimately, I trained for about 700 epochs
# The measure of sparseness was used for validation because
# of the risk of exploding/vanishing gradients
n_epochs = 1000
batch_size = 10
batches = term_matrix.shape[0] // batch_size
for epoch in xrange(n_epochs):
    c = []
    for batch_index in xrange(batches):
        c.append(train_da(batch_index))
    W2_pre = pretrainer.W.get_value(borrow = True)
    output = numpy.dot(term_matrix, W2_pre) 
    output = 1 / (1 + numpy.exp(-output))
    sparseness = numpy.sum(output) / (output.shape[0] * output.shape[1])
    print 'Training epoch %d, cost %f, sparseness %f' % (epoch, numpy.mean(c), sparseness)

In [ ]:
# The trained weight matrix is extracted, and hidden-unit activations
# are extracted and saved to disk for pre-training W1. 
W2_pre = pretrainer.W.get_value(borrow = True)
output = numpy.dot(term_matrix, W2_pre)
savemat("./t1_ntm_pretrain.mat", { 'activations' : output,
                                 'W2' : W2_pre})
(W2_pre.shape, W2_pre[0,:], output.shape)

In [ ]:
#W2_pre = loadmat(to_path + "t1_ntm_pretrain.mat", variable_names = "W2").get("W2").astype("float32")

In [ ]:
# W1 was pretrained in R
# For each document, its pre-trained embedding is the sum of the W1 activations for all terms found in the document
pretrained_W1 = loadmat(to_path + "t1_ntm_pret.mat", variable_names = "w1").get("w1").astype("float32") 

In [ ]:
# The training set is (n_grams, 2 + n_epochs) matrix
# Columns are:
#      0. Index of a document (d_pos)
#      1. Index of a term found in d_pos (g)
#      2...(1 + n_epochs). Indices of randomly selected documents that do not contain g (d_neg)
#         d_negs were selected proportionate to the inverse of the number of terms in each document,
#         so documents get approximately the same number of total passes in each epoch

examples = loadmat(to_path + "t1_ntm_pret.mat", variable_names = "examples").get("examples")
examples = numpy.vstack(tuple([examples[:,(0,1,x)] for x in range(2, examples.shape[1])]))

In [ ]:
(n_docs, n_topics, n_terms, n_total_grams) = (pretrained_W1.shape[0], 
                               pretrained_W1.shape[1], 
                               term_matrix.shape[0], 
                                        examples.shape[0])
(n_docs, n_topics, n_terms, n_total_grams)

In [ ]:
# Basic NTM model
def build_ntm(w1_weights, 
              w2_weights,
              term_matrix,
              W1_regularizer = l2(0.001), 
              W2_regularizer = l2(0.001)
             ):
    
    n_docs = w1_weights.shape[0]
    n_topics = w2_weights.shape[1]
    n_terms = w2_weights.shape[0]
    
    ntm = Graph()
    
    ntm.add_input(name = "d_pos", input_shape = (1,), dtype = "int")
    ntm.add_input(name = "d_neg", input_shape = (1,), dtype = "int")
    ntm.add_shared_node(Embedding(input_dim = n_docs, 
                                  output_dim = n_topics, 
                                  weights = [w1_weights], 
                                  W_regularizer = W1_regularizer,
                                  input_length = 1),
                        name = "topicmatrix",
                        inputs =  ["d_pos", "d_neg"], 
                        outputs = ["wd_pos", "wd_neg"],
                        merge_mode = None)
    ntm.add_node(Flatten(), name = "wd_pos_", input = "wd_pos")
    ntm.add_node(Flatten(), name = "wd_neg_", input = "wd_neg")
    ntm.add_node(Activation("softmax"), name = "ld_pos", input = "wd_pos_")
    ntm.add_node(Activation("softmax"), name = "ld_neg", input = "wd_neg_")
    
    ntm.add_input(name = "g", input_shape = (1,), dtype = "int")
    ntm.add_node(Embedding(input_dim = n_terms, 
                          output_dim = 300,
                          weights = [term_matrix], 
                           trainable = False,
                           input_length = 1), 
                 name = "le", input = "g")
    ntm.add_node(Flatten(), input = "le", name = "le_")
    ntm.add_node(Dense(n_topics, activation = "sigmoid", 
                       weights = [w2_weights, numpy.zeros(n_topics)], 
                       W_regularizer = W2_regularizer),
                 name = "lt", input = "le_")
    
    ntm.add_node(Layer(),
                       name = "ls_pos", 
                       inputs = ["lt", "ld_pos"], 
                       merge_mode = 'dot', dot_axes = -1)
    ntm.add_node(Layer(), 
                       name = "ls_neg", 
                       inputs = ["lt", "ld_neg"], 
                        merge_mode = 'dot', dot_axes = -1)
    return ntm

In [ ]:
# Train the model

# Very large batch sizes are a good idea.  
# Even with very large batches, the its unlikely that many weights in W1 will be 
# triggered more than once each batch.  But, W2 weights get updated from every row. 
# W2 therefore wants to overfit before W1 is finished training. Using large batch
# sizes mitigates the effect. 
batch_size = 20000
n_epochs = 10
margin = 0.5
ntm = build_ntm(
              w1_weights = pretrained_W1, 
              w2_weights = W2_pre,
              term_matrix = term_matrix,
              W1_regularizer = l2(0.001), 
              W2_regularizer = l2(0.001))

def output_shape(input_shape):
    return (None, 1)

def sumLam(x):
    return (margin + (x[1] - x[0]))

summer = LambdaMerge(layers = [ntm.nodes["ls_pos"], ntm.nodes["ls_neg"] ], 
                     function = sumLam,
                     output_shape = output_shape)
ntm.add_node(summer, inputs = ["ls_pos", "ls_neg"], 
             name = "summed")
ntm.add_output(name = "loss_out",  input= "summed")

def rawloss(x_train, x_test):
    return x_train * x_test

# Adadelta tended to converge more quickly than SGD
ntm.compile(loss = {'loss_out' : rawloss},
           optimizer = 'Adadelta') 

checkpointer = ModelCheckpoint(filepath="./checkpointweights.hdf5", verbose = 1, save_best_only=True)

train_data = examples
train_shape = (train_data.shape[0], 1)
g = numpy.reshape(examples[:,1], train_shape)
d_pos = numpy.reshape(examples[:,0], train_shape)
d_neg = numpy.reshape(examples[:,2], train_shape)
        
ntm.fit(data = {
            "g" : g, 
            "d_pos" : d_pos, 
            "d_neg" : d_neg,
            "loss_out" : numpy.reshape(numpy.ones(trainer.shape[0], 
                                                  dtype = theano.config.floatX), train_shape)
        }, callbacks = [checkpointer],
        validation_split = 0.005,
        nb_epoch = n_epochs, 
        batch_size = batch_size)

In [ ]:
json_string = ntm.to_json()
open('ntm_final.json', 'w').write(json_string)
ntm.save_weights(to_path + 'ntm_finalweights_.h5', overwrite=True)

In [ ]:
# sNTM - Not fully tested
n_categories = 3
ntm.add_node(Dense(n_categories, activation = "sigmoid"), input = "ld_pos", name = "ll")
ntm.add_output(name = "label", input = "ll")
ntm.compile(loss = {'loss_out' : threshold,
                   'label' : 'categorical_crossentropy'}, 
           optimizer = "Adadelta")